<a href="https://colab.research.google.com/github/Debajyoti2004/coursera-test/blob/main/Translator_pyng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'hindi-english-parallel-corpus:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F789090%2F1355361%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241011%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241011T090700Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D0821e0179736c1ed7dfe6bc3d996a5b80b3feb6dbc6c656cee2f0bf9b1a92bccc19df775053646fe0bc2c2599383e50227867d3533c31917bae445c0df41eca769e02a5e00498dc0f7f3558c28b42596e6bdb90cb7f106c86ab8e6f614dde89ac61e4f63652c94069087864a1614204161a8296f22c4ce491275710ae25ecab9954099d4af3fe31044179ab243222736e7f6d53c406dbc3154232f245fca6295851441b123db886418bb3ab440fea4a7b240f3d82059bfefb9f976f3282121d1b4d08b282455e2ddbe503dd47a40ed669a5a8ecee928a8a92251b43333f48265e0d15e32891523a3820d716ca15f70aaff75960b364788ddae072cd38b7603f3'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')

[==================================================] 117753358 bytes downloaded
Downloaded and uncompressed: hindi-english-parallel-corpus
Data source import complete.


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/hindi-english-parallel-corpus/hindi_english_parallel.csv


In [3]:
df=pd.read_csv('/kaggle/input/hindi-english-parallel-corpus/hindi_english_parallel.csv')
df.head()

,hindi,english
0,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें,Give your application an accessibility workout
1,एक्सेर्साइसर पहुंचनीयता अन्वेषक,Accerciser Accessibility Explorer
2,निचले पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the bottom panel
3,ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the top panel
4,उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से नि...,A list of plugins that are disabled by default


### Preprocessing Data on Data Frame

In [4]:
no_of_data=500
df_new=df.iloc[:no_of_data]

In [5]:
def clean(row):
    row['hindi']=str(row['hindi']).lower().split()
    row['english']=str(row['english']).lower().split()
    return row
df_new=df_new.apply(clean,axis=1)
df_new.head()


,hindi,english
0,"[अपने, अनुप्रयोग, को, पहुंचनीयता, व्यायाम, का,...","[give, your, application, an, accessibility, w..."
1,"[एक्सेर्साइसर, पहुंचनीयता, अन्वेषक]","[accerciser, accessibility, explorer]"
2,"[निचले, पटल, के, लिए, डिफोल्ट, प्लग-इन, खाका]","[the, default, plugin, layout, for, the, botto..."
3,"[ऊपरी, पटल, के, लिए, डिफोल्ट, प्लग-इन, खाका]","[the, default, plugin, layout, for, the, top, ..."
4,"[उन, प्लग-इनों, की, सूची, जिन्हें, डिफोल्ट, रू...","[a, list, of, plugins, that, are, disabled, by..."


In [6]:
def len_string(row):
  row['hindi_len']=len(row['hindi'])
  row['english_len']=len(row['english'])
  return row
df_new=df_new.apply(len_string,axis=1)
df_new.head()

,hindi,english,hindi_len,english_len
0,"[अपने, अनुप्रयोग, को, पहुंचनीयता, व्यायाम, का,...","[give, your, application, an, accessibility, w...",8,6
1,"[एक्सेर्साइसर, पहुंचनीयता, अन्वेषक]","[accerciser, accessibility, explorer]",3,3
2,"[निचले, पटल, के, लिए, डिफोल्ट, प्लग-इन, खाका]","[the, default, plugin, layout, for, the, botto...",7,8
3,"[ऊपरी, पटल, के, लिए, डिफोल्ट, प्लग-इन, खाका]","[the, default, plugin, layout, for, the, top, ...",7,8
4,"[उन, प्लग-इनों, की, सूची, जिन्हें, डिफोल्ट, रू...","[a, list, of, plugins, that, are, disabled, by...",12,9


In [7]:
def max_len(row):
  if row['hindi_len']>row['english_len']:
    return row['hindi_len']
  else:
    return row['english_len']
df_new['max_len']=df_new.apply(max_len,axis=1)

In [8]:
max_len=df_new['max_len'].max()
max_len

29

In [9]:
max_len_hindi=df_new['hindi_len'].max()
max_len_eng=df_new['english_len'].max()
print("max length of hindi:",max_len_hindi)
print("max length of english:",max_len_eng)

max length of hindi: 29
max length of english: 25


In [10]:
from nltk.stem import SnowballStemmer
stemmer_english=SnowballStemmer('english')

def stemmer(row):
  row['english']=[stemmer_english.stem(word) for word in row['english']]
  return row

df_new=df_new.apply(stemmer,axis=1)
df_new.head()

,hindi,english,hindi_len,english_len,max_len
0,"[अपने, अनुप्रयोग, को, पहुंचनीयता, व्यायाम, का,...","[give, your, applic, an, access, workout]",8,6,8
1,"[एक्सेर्साइसर, पहुंचनीयता, अन्वेषक]","[accercis, access, explor]",3,3,3
2,"[निचले, पटल, के, लिए, डिफोल्ट, प्लग-इन, खाका]","[the, default, plugin, layout, for, the, botto...",7,8,8
3,"[ऊपरी, पटल, के, लिए, डिफोल्ट, प्लग-इन, खाका]","[the, default, plugin, layout, for, the, top, ...",7,8,8
4,"[उन, प्लग-इनों, की, सूची, जिन्हें, डिफोल्ट, रू...","[a, list, of, plugin, that, are, disabl, by, d...",12,9,12


In [11]:
def add_tags(row):
  row['english']=['<start>']+row['english']+['<end>']
  return row

df_new=df_new.apply(add_tags,axis=1)

In [12]:
df_new.head()

,hindi,english,hindi_len,english_len,max_len
0,"[अपने, अनुप्रयोग, को, पहुंचनीयता, व्यायाम, का,...","[<start>, give, your, applic, an, access, work...",8,6,8
1,"[एक्सेर्साइसर, पहुंचनीयता, अन्वेषक]","[<start>, accercis, access, explor, <end>]",3,3,3
2,"[निचले, पटल, के, लिए, डिफोल्ट, प्लग-इन, खाका]","[<start>, the, default, plugin, layout, for, t...",7,8,8
3,"[ऊपरी, पटल, के, लिए, डिफोल्ट, प्लग-इन, खाका]","[<start>, the, default, plugin, layout, for, t...",7,8,8
4,"[उन, प्लग-इनों, की, सूची, जिन्हें, डिफोल्ट, रू...","[<start>, a, list, of, plugin, that, are, disa...",12,9,12


### Tokenization (Custom) on Data frame:

In [13]:
class SimpleTokenizer:
  def __init__(self,max_len_hindi,max_len_eng):
    self.max_len_hindi=max_len_hindi
    self.max_len_eng=max_len_eng
    self.word_idx_hindi={}
    self.idx_word_hindi={}
    self.vocab_size_hindi=0

    self.word_idx_eng={}
    self.idx_word_eng={}
    self.vocab_size_eng=0

  def build_vocab_hindi(self,sentences):
    for sentence in sentences:
      for word in sentence:
        if word not in self.word_idx_hindi:
          self.word_idx_hindi[word]=self.vocab_size_hindi
          self.idx_word_hindi[self.vocab_size_hindi]=word
          self.vocab_size_hindi+=1
    self.word_idx_hindi['<oov>']=self.vocab_size_hindi
    self.idx_word_hindi[self.vocab_size_hindi]='<oov>'
    self.vocab_size_hindi+=1
    return self.vocab_size_hindi,self.word_idx_hindi,self.idx_word_hindi

  def build_vocab_eng(self,sentences):
    for sentence in sentences:
      for word in sentence:
        if word not in self.word_idx_eng:
          self.word_idx_eng[word]=self.vocab_size_eng
          self.idx_word_eng[self.vocab_size_eng]=word
          self.vocab_size_eng+=1

    self.word_idx_eng['<oov>']=self.vocab_size_eng
    self.idx_word_eng[self.vocab_size_eng]='<oov>'
    self.vocab_size_eng+=1
    return self.vocab_size_eng,self.word_idx_eng,self.idx_word_eng

  def encode_hindi(self,sentence):
    encoded=[self.word_idx_hindi[word] for word in sentence]

    if len(encoded)<self.max_len_hindi:
      encoded+=[self.word_idx_hindi['<oov>']] * (self.max_len_hindi-len(encoded))

    else:
      encoded=encoded[:self.max_len_hindi]

    return encoded

  def encode_english(self,sentence):
    encoded=[self.word_idx_eng[word] for word in sentence]
    if len(encoded)<self.max_len_eng:
      encoded+=[self.word_idx_eng['<oov>']] * (self.max_len_eng-len(encoded))

    else:
      encoded=encoded[:self.max_len_eng]

    return encoded

  def decode_hindi(self,sentence):
    return [self.idx_word_hindi[word] for word in sentence]

  def decode_eng(self,sentence):
    return [self.idx_word_eng[word] for word in sentence]

In [14]:
from torch.utils.data import Dataset,DataLoader
from torch.nn.utils.rnn import pad_sequence

In [15]:
simpletokenizer=SimpleTokenizer(max_len_hindi=29,max_len_eng=27)


In [16]:
vocab_size_hindi,word_idx_hindi,idx_word_hindi=simpletokenizer.build_vocab_hindi(df_new['hindi'])
vocab_size_english,word_idx_english,idx_word_english=simpletokenizer.build_vocab_eng(df_new['english'])
print('Hindi Vocab Size:',vocab_size_hindi)
print('English Vocab Size:',vocab_size_english)
oov='<oov>'
st='<start>'
print("start index:",word_idx_hindi[oov])
print("start index:",word_idx_english[st])

print("start:",idx_word_hindi[0])
print("start:",idx_word_english[0])

Hindi Vocab Size: 408
English Vocab Size: 313
start index: 407
start index: 0
start: अपने
start: <start>


In [17]:
def encode_sentence(row):
  row['hindi_encoded']=simpletokenizer.encode_hindi(row['hindi'])
  row['english_encoded']=simpletokenizer.encode_english(row['english'])
  return row

df_new=df_new.apply(encode_sentence,axis=1)
df_new.head()

,hindi,english,hindi_len,english_len,max_len,hindi_encoded,english_encoded
0,"[अपने, अनुप्रयोग, को, पहुंचनीयता, व्यायाम, का,...","[<start>, give, your, applic, an, access, work...",8,6,8,"[0, 1, 2, 3, 4, 5, 6, 7, 407, 407, 407, 407, 4...","[0, 1, 2, 3, 4, 5, 6, 7, 312, 312, 312, 312, 3..."
1,"[एक्सेर्साइसर, पहुंचनीयता, अन्वेषक]","[<start>, accercis, access, explor, <end>]",3,3,3,"[8, 3, 9, 407, 407, 407, 407, 407, 407, 407, 4...","[0, 8, 5, 9, 7, 312, 312, 312, 312, 312, 312, ..."
2,"[निचले, पटल, के, लिए, डिफोल्ट, प्लग-इन, खाका]","[<start>, the, default, plugin, layout, for, t...",7,8,8,"[10, 11, 12, 13, 14, 15, 16, 407, 407, 407, 40...","[0, 10, 11, 12, 13, 14, 10, 15, 16, 7, 312, 31..."
3,"[ऊपरी, पटल, के, लिए, डिफोल्ट, प्लग-इन, खाका]","[<start>, the, default, plugin, layout, for, t...",7,8,8,"[17, 11, 12, 13, 14, 15, 16, 407, 407, 407, 40...","[0, 10, 11, 12, 13, 14, 10, 17, 16, 7, 312, 31..."
4,"[उन, प्लग-इनों, की, सूची, जिन्हें, डिफोल्ट, रू...","[<start>, a, list, of, plugin, that, are, disa...",12,9,12,"[18, 19, 20, 21, 22, 14, 23, 24, 25, 26, 27, 2...","[0, 18, 19, 20, 12, 21, 22, 23, 24, 11, 7, 312..."


### Converting From Data frame to Suitable Data that suits our Model

In [18]:
from torch.nn.functional import one_hot

class CustomDataset(Dataset):
  def __init__(self,df,vocab_size_hindi,vocab_size_english):
    self.df=df
    self.vocab_size_hindi=vocab_size_hindi
    self.vocab_size_english=vocab_size_english

  def __len__(self):
    return len(self.df)

  def __getitem__(self,idx):
    row=self.df.iloc[idx]
    hindi_encoded=torch.tensor(row['hindi_encoded'])
    english_encoded=torch.tensor(row['english_encoded'])
    english_one_hot=one_hot(english_encoded,num_classes=self.vocab_size_english)
    return (hindi_encoded,english_encoded),english_one_hot

In [19]:
from sklearn.model_selection import train_test_split
train_df,test_df=train_test_split(df_new,test_size=0.1,random_state=42)

train_data_loader=DataLoader(CustomDataset(train_df,vocab_size_hindi,vocab_size_english),batch_size=32,shuffle=True)
test_data_loader=DataLoader(CustomDataset(test_df,vocab_size_hindi,vocab_size_english),batch_size=3)

print("length of train data loader:",len(train_data_loader))
print("length of test data loader:",len(test_data_loader))

length of train data loader: 15
length of test data loader: 17


In [20]:
print("checking data number:",len(train_data_loader)*32)
print("checking data number:",len(test_data_loader)*3)

checking data number: 480
checking data number: 51


In [21]:
import torch
from torch import nn
from torch.nn import functional as F

In [22]:
print("GPU available:",torch.cuda.is_available())
print("Number of GPU:",torch.cuda.device_count())
print("GPU name:",torch.cuda.get_device_name())
print("GPU type:",torch.cuda.get_device_capability())

GPU available: True
Number of GPU: 1
GPU name: Tesla T4
GPU type: (7, 5)


In [23]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:",device)

Using device: cuda


### Basic Attention Model(Version 1):

In [24]:
class Pre_LSTM(nn.Module):
  def __init__(self,input_size,d_model,output_size,hidden_size,num_layers):
    super(Pre_LSTM,self).__init__()
    self.embedding=nn.Embedding(input_size,d_model)
    self.lstm=nn.LSTM(d_model,hidden_size,num_layers,batch_first=True,bidirectional=True)

  def forward(self,x):
    x=self.embedding(x)
    output,(hidden_state,cell_state)=self.lstm(x)
    return output,hidden_state,cell_state


In [25]:
class Attention(nn.Module):
  def __init__(self,T_x,hidden_size):
    super(Attention,self).__init__()
    self.T_x=T_x
    self.attn=nn.Linear(hidden_size*2,hidden_size)
    self.Dense=nn.Linear(hidden_size,1)

  def forward(self,a,s_prev):
    s_prev=s_prev.unsqueeze(1).repeat(1,self.T_x,1)
    concat=torch.cat((a,s_prev),dim=2)
    energy=torch.tanh(self.attn(concat))
    attention=self.Dense(energy)

    attention=attention.squeeze(2)
    alphas=F.softmax(attention,dim=1)

    context=torch.bmm(alphas.unsqueeze(1),a).squeeze(1)

    return context

In [26]:
class Attention_Model(nn.Module):
    def __init__(self, T_x, T_y, input_size, d_model, encoder_num_layers, encoder_hidden_size, decoder_num_layers, output_size):
        super(Attention_Model, self).__init__()
        self.T_x = T_x
        self.T_y = T_y
        self.Pre_lstm = Pre_LSTM(input_size, d_model, input_size, encoder_hidden_size, encoder_num_layers)
        self.Attention = Attention(T_x, encoder_hidden_size * 2)
        self.Post_lstm = nn.LSTM(encoder_hidden_size * 2, encoder_hidden_size * 2, decoder_num_layers, batch_first=True)
        self.fc = nn.Linear(encoder_hidden_size * 2, output_size)
        self.softmax = nn.Softmax(dim=2)
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    def forward(self, input, target_tensor=None):
        a, _, _ = self.Pre_lstm(input)

        s_prev = torch.randn(a.size(0), a.size(2), device=self.device) * 1e-5
        c_prev = torch.randn(a.size(0), a.size(2), device=self.device) * 1e-5

        outputs = []

        for i in range(self.T_y):
            context = self.Attention(a, s_prev)
            context = context.unsqueeze(1)

            output, (s_prev, c_prev) = self.Post_lstm(context, (s_prev.unsqueeze(0), c_prev.unsqueeze(0)))
            s_prev=s_prev.squeeze(0)
            c_prev=c_prev.squeeze(0)

            output = self.fc(output)
            output = self.softmax(output)

            outputs.append(output)

        return torch.cat(outputs, dim=1)

### Training and Testing Basic Attention Model(Version 1):

In [27]:
class Attention_model_train_test:
    def __init__(self, vocab_size_hindi, vocab_size_english, max_len_hindi,max_len_eng):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.vocab_size_hindi = vocab_size_hindi
        self.vocab_size_english = vocab_size_english
        self.model = Attention_Model(max_len_hindi, max_len_eng, vocab_size_hindi, 512, 2, 512, 1, vocab_size_english).to(self.device)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=0.001)
        self.criterion = nn.BCELoss()

    def train(self, train_data_loader, epochs=10):
        self.model.train()
        for epoch in range(epochs):
            total_loss = 0
            for (hindi_encoded, english_encoded), english_one_hot in train_data_loader:
                hindi_encoded, english_encoded, english_one_hot = hindi_encoded.to(self.device), english_encoded.to(self.device), english_one_hot.to(self.device)

                self.optimizer.zero_grad()

                output = self.model(hindi_encoded, english_one_hot)

                loss = self.criterion(output, english_one_hot.float())

                loss.backward()
                self.optimizer.step()

                total_loss += loss.item()

            print(f'Epoch {epoch + 1}/{epochs}, Train Loss: {total_loss / len(train_data_loader)}')

        return self.model

    def test(self, test_data_loader):
        self.model.eval()
        total_loss = 0
        with torch.no_grad():
            for (hindi_encoded, english_encoded), english_one_hot in test_data_loader:
                hindi_encoded, english_encoded, english_one_hot = hindi_encoded.to(self.device), english_encoded.to(self.device), english_one_hot.to(self.device)

                output = self.model(hindi_encoded, english_one_hot)

                loss = self.criterion(output, english_one_hot.float())
                total_loss += loss.item()

        print(f'Test Loss: {total_loss / len(test_data_loader)}')


In [28]:
Attention_Model_train_test=Attention_model_train_test(vocab_size_hindi,vocab_size_english,max_len_hindi=29,max_len_eng=27)
Attention_model_Trained=Attention_Model_train_test.train(train_data_loader,epochs=10)

Epoch 1/10, Train Loss: 0.0066734456767638525
Epoch 2/10, Train Loss: 0.003879220566401879
Epoch 3/10, Train Loss: 0.003552910095701615
Epoch 4/10, Train Loss: 0.0028341531132658323
Epoch 5/10, Train Loss: 0.0026682208835457763
Epoch 6/10, Train Loss: 0.0025652461374799413
Epoch 7/10, Train Loss: 0.0024092426057904957
Epoch 8/10, Train Loss: 0.002320452632072071
Epoch 9/10, Train Loss: 0.0021715133062874276
Epoch 10/10, Train Loss: 0.002168302439774076


In [29]:
Attention_Model_train_test.test(test_data_loader)

Test Loss: 0.002345048456096693


### Self Attention Model(Version 2):

Using following Layer:
1.   Embedding Layer
2.   Positional Encoding
3.   MultiAttention Model
4.   Feed Forward Layer
5.   Softmax



In [30]:
def get_angles(pos, i, d_model):
    angle_rates = 1 / torch.exp((2 * (i // 2)) * torch.log(torch.tensor(10000.0)) / d_model)
    return pos * angle_rates

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_len, device):
        super(PositionalEncoding, self).__init__()
        self.d_model = d_model
        self.max_seq_len = max_seq_len
        pe = torch.zeros(max_seq_len, d_model)
        position = torch.arange(0, max_seq_len, dtype=torch.float).unsqueeze(1)
        div_term = get_angles(position, torch.arange(0, d_model, 2).unsqueeze(0), d_model)
        pe[:, 0::2] = torch.sin(div_term)
        pe[:, 1::2] = torch.cos(div_term)
        self.pe = pe.to(device)

    def forward(self, x):
        return x + self.pe[:x.size(1), :]

In [31]:
class Transformer_model(nn.Module):
  def __init__(self,input_size,max_len_hindi,max_len_eng,d_model,encoder_num_layers,encoder_hidden_size,decoder_num_layers,output_size, device):
    super(Transformer_model,self).__init__()
    self.embedding_src=nn.Embedding(input_size,d_model)
    self.embedding_trg=nn.Embedding(output_size,d_model)
    self.positional_encoding_hindi=PositionalEncoding(d_model,max_len_hindi, device)
    self.positional_encoding_english=PositionalEncoding(d_model,max_len_eng, device)
    self.transformer=nn.Transformer(d_model=d_model,nhead=8,num_encoder_layers=encoder_num_layers,num_decoder_layers=decoder_num_layers,
                                    dim_feedforward=2048,dropout=0.1,batch_first=True)
    self.fc=nn.Linear(d_model,output_size)
    self.softmax=nn.Softmax(dim=2)

  def forward(self,src,trg):
    src=self.embedding_src(src)
    trg=self.embedding_trg(trg)
    src=self.positional_encoding_hindi(src)
    trg=self.positional_encoding_english(trg)

    output=self.transformer(src,trg)
    output=self.fc(output)
    output=self.softmax(output)
    return output

### Training and testing Transformer Model:

In [32]:
class Transformer_model_train_test:
    def __init__(self, vocab_size_hindi, vocab_size_english, max_len_hindi,max_len_eng):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.vocab_size_hindi = vocab_size_hindi
        self.vocab_size_english = vocab_size_english
        self.model = Transformer_model(vocab_size_hindi, max_len_hindi,max_len_eng, 512, 4, 512, 3, vocab_size_english,self.device).to(self.device)
        self.criterion = nn.BCELoss()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=0.001)

    def train(self, train_data_loader, epochs=10):
        self.model.train()
        for epoch in range(epochs):
            total_loss = 0
            for (hindi_encoded, english_encoded), english_one_hot in train_data_loader:
                # Move tensors to the appropriate device
                hindi_encoded, english_encoded, english_one_hot = hindi_encoded.to(self.device), english_encoded.to(self.device), english_one_hot.to(self.device)

                self.optimizer.zero_grad()

                output = self.model(hindi_encoded, english_encoded)
                loss = self.criterion(output, english_one_hot.float())
                loss.backward()
                self.optimizer.step()

                total_loss += loss.item()

            print(f'Epoch {epoch + 1}/{epochs}, Train Loss: {total_loss / len(train_data_loader)}')

        return self.model

    def test(self, test_data_loader):
        self.model.eval()
        total_loss = 0
        with torch.no_grad():
            for (hindi_encoded, english_encoded), english_one_hot in test_data_loader:
                # Move tensors to the appropriate device
                hindi_encoded, english_encoded, english_one_hot = hindi_encoded.to(self.device), english_encoded.to(self.device), english_one_hot.to(self.device)

                output = self.model(hindi_encoded, english_encoded)
                loss = self.criterion(output, english_one_hot.float())
                total_loss += loss.item()

            print(f'Test Loss: {total_loss / len(test_data_loader)}')


In [33]:
Transformer_Model_train_and_test=Transformer_model_train_test(vocab_size_hindi,vocab_size_english,max_len_hindi=29,max_len_eng=27)
Transformer_model_Trained=Transformer_Model_train_and_test.train(train_data_loader,epochs=8)

Epoch 1/8, Train Loss: 0.007841769295434156
Epoch 2/8, Train Loss: 0.005657292902469635
Epoch 3/8, Train Loss: 0.006129555869847536
Epoch 4/8, Train Loss: 0.005594307308395704
Epoch 5/8, Train Loss: 0.005621907891084751
Epoch 6/8, Train Loss: 0.005598886994024118
Epoch 7/8, Train Loss: 0.005488331274439891
Epoch 8/8, Train Loss: 0.005515568951765696


In [34]:
Transformer_Model_train_and_test.test(test_data_loader)

Test Loss: 0.005768141544917051


### Seq2Seq model (translator version 2):

In [48]:
class Encoder_BiLSTM(nn.Module):
    def __init__(self, input_size, d_model, hidden_size, num_layers):
        super(Encoder_BiLSTM, self).__init__()
        self.num_layers=num_layers
        self.embedding = nn.Embedding(input_size, d_model)
        self.Bilstm = nn.LSTM(d_model, hidden_size, num_layers, batch_first=True, bidirectional=True)

    def forward(self, x):
        x = self.embedding(x)
        output, (hidden_state, cell_state) = self.Bilstm(x)

        last_hidden = hidden_state.view(self.num_layers, 2, x.size(0), hidden_state.size(2))
        last_cell = cell_state.view(self.num_layers, 2, x.size(0), cell_state.size(2))

        last_hidden = torch.cat((last_hidden[:, 0, :, :], last_hidden[:, 1, :, :]), dim=2)
        last_cell = torch.cat((last_cell[:, 0, :, :], last_cell[:, 1, :, :]), dim=2)

        last_hidden=last_hidden[-2:,:,:]
        last_cell=last_cell[-2:,:,:]
        return output, last_hidden, last_cell

In [49]:
class Decoder_LSTM(nn.Module):
    def __init__(self, input_size, d_model, output_size, hidden_size, num_layers):
        super(Decoder_LSTM, self).__init__()
        self.embedding = nn.Embedding(input_size, d_model)
        self.lstm = nn.LSTM(d_model, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Softmax(dim=2)

    def forward_step(self, input, hidden_state, cell_state):
        input = self.embedding(input)
        output, (hidden_state, cell_state) = self.lstm(input, (hidden_state, cell_state))
        output = self.fc(output)
        output = self.softmax(output)
        return output, hidden_state, cell_state

    def forward(self, x, hidden_state, cell_state, target_tensor=None):
        batch_size = x.size(0)
        seq_len = x.size(1)
        outputs = []

        if target_tensor is None:
            decoder_input = torch.zeros((batch_size, 1), dtype=torch.long).to(x.device)
        else:
            decoder_input = target_tensor[:, 0].unsqueeze(1)

        for i in range(seq_len):
            output, hidden_state, cell_state = self.forward_step(decoder_input, hidden_state, cell_state)
            outputs.append(output)

            if target_tensor is not None and i+1 < seq_len:
                decoder_input = target_tensor[:, i+1].unsqueeze(1)
            else:
                _, top_index = output.max(dim=2)
                decoder_input = top_index.detach()

        outputs = torch.cat(outputs, dim=1)
        return outputs



In [50]:
class seq2seq(nn.Module):
  def __init__(self,input_size,d_model,encoder_num_layers,encoder_hidden_size,decoder_num_layers,decoder_hidden_size,output_size):
    super(seq2seq,self).__init__()
    self.Encoder=Encoder_BiLSTM(input_size,d_model,encoder_hidden_size,encoder_num_layers)
    self.Decoder=Decoder_LSTM(input_size,d_model,output_size,decoder_hidden_size,decoder_num_layers)

  def forward(self,src,trg,target_tensor=None):
    _,hidden_state,cell_state=self.Encoder(src)
    output=self.Decoder(trg,hidden_state,cell_state,target_tensor=target_tensor)
    return output

### Training and testing Seq2Seq Model:

In [51]:
class Seq2Seq_train_and_test:
    def __init__(self, vocab_size_hindi, vocab_size_english, max_len):
        self.vocab_size_hindi = vocab_size_hindi
        self.vocab_size_english = vocab_size_english
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = seq2seq(vocab_size_hindi, 512, 3, 1024, 2, 2048, vocab_size_english).to(self.device)
        self.criterion = nn.BCELoss()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=0.001)

    def train(self, train_data_loader, epochs=10):
        self.model.train()
        for epoch in range(epochs):
            total_loss = 0
            for (hindi_encoded, english_encoded), english_one_hot in train_data_loader:
                hindi_encoded = hindi_encoded.to(self.device)
                english_encoded = english_encoded.to(self.device)
                english_one_hot = english_one_hot.to(self.device)

                self.optimizer.zero_grad()
                output = self.model(hindi_encoded, english_encoded, target_tensor=english_encoded)
                loss = self.criterion(output, english_one_hot.float())
                loss.backward()
                self.optimizer.step()

                total_loss += loss.item()

            print(f'Epoch {epoch+1}/{epochs}, Train Loss: {total_loss/len(train_data_loader)}')

        return self.model

    def test(self, test_data_loader):
        self.model.eval()
        test_loss = 0
        with torch.no_grad():
            for (hindi_encoded, english_encoded), english_one_hot in test_data_loader:
                hindi_encoded = hindi_encoded.to(self.device)
                english_encoded = english_encoded.type(torch.LongTensor).to(self.device)  # Ensure this is LongTensor
                english_one_hot = english_one_hot.to(self.device)

                output = self.model(hindi_encoded, english_encoded, target_tensor=None)
                loss = self.criterion(output, english_one_hot.float())
                test_loss += loss.item()

        print(f'Test Loss: {test_loss/len(test_data_loader)}')


In [52]:
seq2seq_train_and_test=Seq2Seq_train_and_test(vocab_size_hindi,vocab_size_english,max_len=29)
Seq2Seq_model_Trained=seq2seq_train_and_test.train(train_data_loader,epochs=10)

Epoch 1/10, Train Loss: 0.004895123839378357
Epoch 2/10, Train Loss: 0.002142549678683281
Epoch 3/10, Train Loss: 0.0018355132235834996
Epoch 4/10, Train Loss: 0.0014642687126373251
Epoch 5/10, Train Loss: 0.00116975714918226
Epoch 6/10, Train Loss: 0.00093617495149374
Epoch 7/10, Train Loss: 0.0006421778195848068
Epoch 8/10, Train Loss: 0.000499998772284016
Epoch 9/10, Train Loss: 0.00035690636141225693
Epoch 10/10, Train Loss: 0.00022406155912904068


In [53]:
seq2seq_train_and_test.test(test_data_loader)

Test Loss: 0.04553327534128638


In [54]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=e25072ca7dbb2be027f480b0f5f44ad340e3f724c1d2e4c4f237ecbee6859f7d
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


### Comparing Models to find which  model is more suitable for translating:
Using:
1.  Bleu_score
2.  Rouge_score


In [65]:
from nltk.translate.bleu_score import sentence_bleu,SmoothingFunction
import rouge_score
from rouge_score import rouge_scorer

def compute_bleu_score(reference, candidate):
    smoothing_function=SmoothingFunction().method1
    return sentence_bleu([reference], candidate,smoothing_function=smoothing_function)

def compute_rouge_score(reference, candidate):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    reference=' '.join(reference)
    candidate=' '.join(candidate)

    scores = scorer.score(reference, candidate)
    return scores

# Test Data
reference = ['the', 'cat', 'is', 'on', 'the', 'mat']
candidate = ['the', 'cat', 'is', 'on', 'the', 'mat']

# Compute BLEU Score
bleu_score = compute_bleu_score(reference, candidate)
print(f"BLEU Score: {bleu_score}")

# Compute ROUGE Score
rouge_scores = compute_rouge_score(reference,candidate)
print(f"ROUGE Scores: {rouge_scores}")

BLEU Score: 1.0
ROUGE Scores: {'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0), 'rouge2': Score(precision=1.0, recall=1.0, fmeasure=1.0), 'rougeL': Score(precision=1.0, recall=1.0, fmeasure=1.0)}


In [66]:
from collections import Counter
import math
import numpy as np

def count(sequence,n):
    return Counter(tuple(sequence[i:i+n]) for i in range(len(sequence)-n+1))

def clip_count(candidate_count,reference_count):
    clip_counts=Counter()
    for candidate_n_gram,candidate_count in candidate_count.items():
        if candidate_n_gram in reference_count:
            clip_counts[candidate_n_gram]=min(candidate_count,reference_count[candidate_n_gram])
    return clip_counts

def bravity_penalty(candidate_length,reference_length):
    if candidate_length>reference_length:
        return 1
    else:
        return math.exp(1-(reference_length/candidate_length))

In [67]:
def Bleu_score(references,candidate,n):
  candidate_length=len(candidate)
  references_length=[len(reference) for reference in references]
  references_length=max(references_length)

  references_count=[Counter() for _ in range(n)]
  for ref in references:
    for i in range(n):
      references_count[i]+=count(ref,i+1)

  candidate_counts=[count(candidate,i) for i in range(1,n+1)]
  precision=[]
  for i in range(n):
    clip_counts=clip_count(candidate_counts[i],references_count[i])
    precision.append(sum(clip_counts.values())/sum(candidate_counts[i].values()))

  bp=bravity_penalty(candidate_length,references_length)
  bleu_score=bp*np.exp(np.mean(np.log(precision)))
  return bleu_score

In [68]:
if __name__=='__main__':
  candidate=["I","am","going","to","school"]
  references=[["I","am","going","to","school"],["I","am","going","to","the","school"],["I","will","go","to","the","school"]]
  Custom_bleu_score=Bleu_score(references,candidate,n=4)
  in_build_bleu_score=sentence_bleu(references,candidate,smoothing_function=SmoothingFunction().method1)
  print(f'Custom_bleu_score: {Custom_bleu_score}')
  print(f'In_build_bleu_score: {in_build_bleu_score}')


Custom_bleu_score: 0.8187307530779819
In_build_bleu_score: 1.0


In [69]:
def text_generator_attention_model(df, idx):
    row = df.iloc[idx]
    hindi_sentence = row['hindi']
    english_sentence = row['english']
    input_tensor = torch.tensor(row['hindi_encoded']).unsqueeze(0)

    device = next(Attention_model_Trained.parameters()).device
    input_tensor = input_tensor.to(device)

    # Get the model's output
    output = Attention_model_Trained(input_tensor, None)
    output = output.argmax(dim=2)
    predicted_sentence = simpletokenizer.decode_eng(output.squeeze(0).tolist())

    print("===Predicted Sentence===")
    print(' '.join(predicted_sentence))

    print("===Actual Sentence===")
    print(' '.join(english_sentence))

    return predicted_sentence, english_sentence

In [71]:
idx=10
predicted_sentence, english_sentence=text_generator_attention_model(df_new, idx)
print(f'In Build bleu_score: {compute_bleu_score(english_sentence,predicted_sentence)}')
print(f'Rouge_score: {compute_rouge_score(english_sentence,predicted_sentence)}')
print(f'(Custom made) Bleu score:{Bleu_score(english_sentence,predicted_sentence,n=4)}')


===Predicted Sentence===
<start> the s the the the the <end> <end> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov>
===Actual Sentence===
<start> the color and opac of the highlight fill. <end>
In Build bleu_score: 0.017554679594930896
Rouge_score: {'rouge1': Score(precision=0.14814814814814814, recall=0.4, fmeasure=0.21621621621621623), 'rouge2': Score(precision=0.038461538461538464, recall=0.1111111111111111, fmeasure=0.05714285714285715), 'rougeL': Score(precision=0.14814814814814814, recall=0.4, fmeasure=0.21621621621621623)}
(Custom made) Bleu score:0.0


<ipython-input-67-5cd53324ae30>:18: RuntimeWarning: divide by zero encountered in log
  bleu_score=bp*np.exp(np.mean(np.log(precision)))


In [74]:
def text_generator_Transformer(df, word_to_idx, idx):
    row = df.iloc[idx]
    hindi_sentence = row['hindi']
    english_sentence = row['english']
    input_tensor = torch.tensor(row['hindi_encoded']).unsqueeze(0)
    english_encoded = torch.tensor(row['english_encoded']).unsqueeze(0)

    device = next(Transformer_model_Trained.parameters()).device
    encoder_input = input_tensor.to(device)
    decoder_input = torch.tensor([word_to_idx['<start>']]).unsqueeze(0).to(device)

    outputs = []

    for _ in range(english_encoded.shape[1]):
        output = Transformer_model_Trained(encoder_input, decoder_input)
        _, top_index = output.max(dim=2)
        outputs.append(top_index.item())
        decoder_input = top_index.detach()

    outputs = torch.tensor(outputs).to(device)

    predicted_sentence = simpletokenizer.decode_eng(outputs.tolist())

    print("===Predicted Sentence===")
    print(' '.join(predicted_sentence))

    print("===Actual Sentence===")
    print(' '.join(english_sentence))

    return predicted_sentence, english_sentence


In [75]:
idx=10
word_to_idx=simpletokenizer.word_idx_eng
idx_to_word=simpletokenizer.idx_word_eng
predicted_sentence, english_sentence=text_generator_Transformer(df_new, word_to_idx, idx)
print(f'In Build bleu_score: {compute_bleu_score(english_sentence,predicted_sentence)}')
print(f'Rouge_score: {compute_rouge_score(english_sentence,predicted_sentence)}')
print(f'(Custom made) Bleu score:{Bleu_score(english_sentence,predicted_sentence,n=4)}')

===Predicted Sentence===
<oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov> <oov>
===Actual Sentence===
<start> the color and opac of the highlight fill. <end>
In Build bleu_score: 0
Rouge_score: {'rouge1': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.0, recall=0.0, fmeasure=0.0)}
(Custom made) Bleu score:0.0


<ipython-input-67-5cd53324ae30>:18: RuntimeWarning: divide by zero encountered in log
  bleu_score=bp*np.exp(np.mean(np.log(precision)))


In [78]:
def text_generator_seq2seq(df,idx):
  row=df.iloc[idx]
  hindi_sentence=row['hindi']
  english_sentence=row['english']
  input_tensor=torch.tensor(row['hindi_encoded']).unsqueeze(0)
  english_encoded=torch.tensor(row['english_encoded']).unsqueeze(0)

  device = next(seq2seq_train_and_test.model.parameters()).device
  encoder_input=input_tensor.to(device)
  english_encoded = english_encoded.to(device)

  output=Seq2Seq_model_Trained(encoder_input,english_encoded,target_tensor=None)
  output=output.argmax(dim=2)
  predicted_sentence=simpletokenizer.decode_eng(output.squeeze(0).tolist())

  print("===Predicted Sentence===")
  print(' '.join(predicted_sentence))

  print("===Actual Sentence===")
  print(' '.join(english_sentence))
  return predicted_sentence, english_sentence

In [79]:
idx=10
predicted_sentence, english_sentence=text_generator_seq2seq(df_new,idx)
print(f'In Build bleu_score: {compute_bleu_score(english_sentence,predicted_sentence)}')
print(f'Rouge_score: {compute_rouge_score(english_sentence,predicted_sentence)}')
print(f'(Custom made) Bleu score:{Bleu_score(english_sentence,predicted_sentence,n=4)}')

===Predicted Sentence===
<start> bookmark bookmark has tabl tabl tabl tabl tabl tabl tabl tabl tabl tabl tabl tabl tabl tabl tabl tabl tabl tabl tabl tabl tabl tabl tabl
===Actual Sentence===
<start> the color and opac of the highlight fill. <end>
In Build bleu_score: 0.006980361417366381
Rouge_score: {'rouge1': Score(precision=0.037037037037037035, recall=0.1, fmeasure=0.05405405405405406), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.037037037037037035, recall=0.1, fmeasure=0.05405405405405406)}
(Custom made) Bleu score:0.0


<ipython-input-67-5cd53324ae30>:18: RuntimeWarning: divide by zero encountered in log
  bleu_score=bp*np.exp(np.mean(np.log(precision)))


### Usually Transformer model most suitable let's see

In [80]:
def compare_models(models, test_data_loader):
    bleu_scores=[]
    for model in models:
        model.eval()
        with torch.no_grad():
            bleu_score=0
            rouge_score=0
            for (hindi_encoded, english_encoded), english_one_hot in test_data_loader:
                output = model.forward(hindi_encoded, english_encoded)
                output=output.argmax(dim=2)
                predicted=simpletokenizer.decode(output.tolist())
                reference=simpletokenizer.decode(english_encoded.tolist())
                bleu_score+=compute_bleu_score(reference,predicted)
                rouge_score+=compute_rouge_score(reference,predicted)

            bleu_scores.append(bleu_score/len(test_data_loader))
            rouge_scores.append(rouge_score/len(test_data_loader))

    return bleu_scores,rouge_scores


In [81]:
class Beam_search_model(nn.Module):
    def __init__(self, model, beam_width, word_to_idx, idx_to_word, vocab_size_english, max_len):
        super(Beam_search_model, self).__init__()
        self.max_len = max_len
        self.word_to_idx = word_to_idx
        self.idx_to_word = idx_to_word
        self.model = model
        self.beam_width = beam_width

    def Transformers_beam_search(self, src, start_token):
        input_seq = [start_token]
        seq = torch.tensor([self.word_to_idx[start_token]]).unsqueeze(0).to(src.device)

        beams = [(input_seq, seq, 0)]
        for i in range(self.max_len):
            new_beams = []
            for beam in beams:
                input_seq, seq, score = beam

                output = self.model(src, seq)
                output = output.squeeze(0)

                output_step = output[i, :]

                top_k_probs, top_k_indices = torch.topk(F.softmax(output_step, dim=-1), k=self.beam_width)

                for prob, idx in zip(top_k_probs, top_k_indices):
                    word = self.idx_to_word[idx.item()]
                    new_input_seq = input_seq + [word]
                    new_seq = torch.cat([seq, idx.unsqueeze(0).unsqueeze(0)], dim=1)
                    new_score = score - torch.log(prob)
                    new_beams.append((new_input_seq, new_seq, new_score))

            beams = sorted(new_beams, key=lambda x: x[2])[:self.beam_width]

        return beams[0]

    def Seq2Seq_beam_search(self, src, start_token):
        input_seq = [start_token]
        seq = torch.tensor([self.word_to_idx[start_token]]).unsqueeze(0).to(src.device)

        beams = [(input_seq, seq, 0)]
        for i in range(self.max_len):
            new_beams = []
            for beam in beams:
                input_seq, seq, score = beam

                output = self.model(src, seq)
                output_step = output[:, i, :]

                top_k_probs, top_k_indices = torch.topk(F.softmax(output_step, dim=-1), k=self.beam_width)

                for prob, idx in zip(top_k_probs.squeeze(0), top_k_indices.squeeze(0)):
                    word = self.idx_to_word[idx.item()]
                    new_input_seq = input_seq + [word]
                    new_seq = torch.cat([seq, idx.unsqueeze(0).unsqueeze(0)], dim=1)
                    new_score = score - torch.log(prob)
                    new_beams.append((new_input_seq, new_seq, new_score))

            beams = sorted(new_beams, key=lambda x: x[2])[:self.beam_width]

        return beams[0]